In [1]:
!pip install pandarallel
import pandas as pd
pd.set_option("max_colwidth", 600)
from bs4 import BeautifulSoup
import requests

from pandarallel import pandarallel


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for pandarallel: filename=pandarallel-1.6.2-py3-none-any.whl size=16411 sha256=b25ddf847697a3b778b6c98ae32d45497dddc8e755e3b16c89fafd7e7184aa76
  Stored in directory: /root/.cache/pip/wheels/01/8d/99/4ce6a1e914ed6ec383bc099dfbe0112dfde0bc39957ca93ec1
Successfully built pandarallel


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [134]:
def extract_urls_topic(url):
    try:
      coverpage = requests.get(url).content
      soup = BeautifulSoup(coverpage, 'html.parser')
     
      rows = []
      coverpage=soup.find_all('div',class_='col-xs-12 col-8')
      coverpage=coverpage[0].find_all('header')
      for i in range(0,len(coverpage)):
        j=1
        for link in coverpage[i].find_all('a'):
          if j==2 :
            rows.append('https://www.eldia.es/'+link.get('href')) 
          j=j+1 
    except:
      rows = 'error'
      
    return rows

In [138]:
extract_urls_topic('https://www.eldia.es/tags/fuerteventura/pagina-3')

['https://www.eldia.es//sociedad/2022/07/11/canarias-suma-casos-nuevos-viruela-68212076.html',
 'https://www.eldia.es//sucesos/sucesos-en-canarias/2022/07/10/muere-kitesurf-fuerteventura-68183464.html',
 'https://www.eldia.es//sociedad/2022/07/10/calor-sigue-dar-tregua-canarias-68179280.html',
 'https://www.eldia.es//canarias/2022/07/07/maxorata-untado-pimenton-elegido-mejor-68085493.html',
 'https://www.eldia.es//sociedad/2022/07/04/hospitales-camaleon-ayudaran-aliviar-presion-67968344.html',
 'https://www.eldia.es//sucesos/sucesos-en-canarias/2022/07/03/llegan-fuerteventura-112-migrantes-rescatados-67925989.html',
 'https://www.eldia.es//canarias/2022/07/02/localizadas-neumaticas-cien-inmigrantes-sur-67915691.html',
 'https://www.eldia.es//sociedad/2022/07/02/intervalos-nubosos-norte-canarias-domingo-67909388.html',
 'https://www.eldia.es//sociedad/2022/07/01/seguiran-lluvias-sabado-canarias-suben-67881333.html']

In [139]:
pandarallel.initialize(nb_workers= 30, progress_bar=True)
df = pd.DataFrame(range(1,6669),columns = ['page'])
url_str=''


for i in range(1,541) :
    url_str='https://www.eldia.es/tags/gran-canaria/pagina-'+str(i)
    df.page[i] = url_str

for i in range(1,188) :
    url_str='https://www.eldia.es/tags/lanzarote/pagina-'+str(i)
    df.page[i+540] = url_str

for i in range(1,111) :
    url_str='https://www.eldia.es/tags/fuerteventura/pagina-'+str(i)
    df.page[i+727] = url_str

for i in range(1,1550) :
    url_str='https://www.eldia.es/internacional/pagina-'+str(i)
    df.page[i+837] = url_str    

for i in range(1,2611) :
    url_str='https://www.eldia.es/nacional/pagina-'+str(i)
    df.page[i+2386] = url_str    

for i in range(1,1213) :
    url_str='https://www.eldia.es/tenerife/pagina-'+str(i)
    df.page[i+4996] = url_str   

for i in range(1,458) :
    url_str='https://www.eldia.es/la-palma/pagina-'+str(i)
    df.page[i+6208] = url_str    

df.page[6666] = 'https://www.eldia.es/la-gomera/'   
df.page[6667] = 'https://www.eldia.es/el-hierro/'    

print(len(df))
df['rows']=df['page'].parallel_apply(extract_urls_topic)
df = df[df.rows.str.len()!=0].reset_index(drop=True)
df = df.explode('rows', ignore_index=True)
df=df.drop_duplicates(subset='rows')
df = df.dropna()
print(df)
df.to_csv('/content/drive/MyDrive/Colab Notebooks/eldia.es/url.csv',index=False)

INFO: Pandarallel will run on 30 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
6668


                                                  page  \
0                                                    1   
1      https://www.eldia.es/tags/gran-canaria/pagina-1   
2      https://www.eldia.es/tags/gran-canaria/pagina-1   
3      https://www.eldia.es/tags/gran-canaria/pagina-1   
4      https://www.eldia.es/tags/gran-canaria/pagina-1   
...                                                ...   
46620                  https://www.eldia.es/el-hierro/   
46621                  https://www.eldia.es/el-hierro/   
46622                  https://www.eldia.es/el-hierro/   
46623                  https://www.eldia.es/el-hierro/   
46624                  https://www.eldia.es/el-hierro/   

                                                                                                      rows  
0                                                                                                    error  
1         https://www.eldia.es//sociedad/2022/08/03/alga-invasora-haciendo-estragos-e